In [1]:
from bs4 import BeautifulSoup
import requests
import time
from urllib.request import urlopen
import ssl
import re
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from os.path import splitext, basename

In [5]:
url='https://www.espncricinfo.com/records/tournament/averages-bowling/icc-cricket-world-cup-2023-24-15338'

r= requests.get(url)

In [6]:
r

<Response [200]>

In [7]:
soup= BeautifulSoup(r.text,"html.parser")

table = soup.find("table", class_=lambda x: x and "ds-table" in x)


In [8]:
print(table)

<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide"><thead class="ds-bg-fill-content-alternate ds-text-left"><tr class=""><td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Player</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Span</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Mat</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Inns</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Balls</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Mdns</span></div></td><td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-curso

In [9]:
header = table.find("thead")

In [10]:
header.text

'PlayerSpanMatInnsBallsMdnsRunsWktsBBIAveEconSR510CtSt'

In [11]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    print(rows[i])

<td class="ds-min-w-max"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Player</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Span</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Mat</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Inns</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Balls</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Mdns</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Runs</span></div></td>
<td class="ds-min-w-max ds-text-right"><div class="ds-popper-wrapper"><span class="ds-cursor-pointer">Wkts</span></div></td>
<td class="d

In [12]:
rows=header.find_all("td")

head=[]
for i in range(len(rows)):
    
    soup1=rows[i]
    x=soup1.find("span",class_="ds-cursor-pointer")
    print(x)
    head.append(x.text)

<span class="ds-cursor-pointer">Player</span>
<span class="ds-cursor-pointer">Span</span>
<span class="ds-cursor-pointer">Mat</span>
<span class="ds-cursor-pointer">Inns</span>
<span class="ds-cursor-pointer">Balls</span>
<span class="ds-cursor-pointer">Mdns</span>
<span class="ds-cursor-pointer">Runs</span>
<span class="ds-cursor-pointer">Wkts</span>
<span class="ds-cursor-pointer">BBI</span>
<span class="ds-cursor-pointer">Ave</span>
<span class="ds-cursor-pointer">Econ</span>
<span class="ds-cursor-pointer">SR</span>
<span class="ds-cursor-pointer">5</span>
<span class="ds-cursor-pointer">10</span>
<span class="ds-cursor-pointer">Ct</span>
<span class="ds-cursor-pointer">St</span>


In [13]:
head

['Player',
 'Span',
 'Mat',
 'Inns',
 'Balls',
 'Mdns',
 'Runs',
 'Wkts',
 'BBI',
 'Ave',
 'Econ',
 'SR',
 '5',
 '10',
 'Ct',
 'St']

In [ ]:
# body = table.find("tbody")
# rows = body.find_all("tr", class_=["ds-table-row-compact-bottom", 'ds-table-row-compact-top'])

# data = []

# for i in range(len(rows)):  

#     temp = []
#     if rows[i]:
#         tdata = rows[i].find_all("td", class_="ds-min-w-max")
#         for j in tdata:
#             temp.append(j.text)

#         if 'ds-table-row-compact-top' in rows[i].get('class', []):
#             if data and temp:
#                 data[-1].append(temp[0])
#             # Add an empty list to represent a separation between rows
#             data.append([])
#         elif data:
#             # Add the player's data to the last list in data
#             data[-1].extend(temp)

# print(data)


In [14]:
body=table.find("tbody")
rows = body.select("tr.ds-table-row-compact-bottom")

data = []
for row in rows:
    temp = []
    tdata = row.find_all("td", class_="ds-min-w-max")
    for i in range(len(tdata)):
        # print(tdata[i].text)
        if i==0 and tdata[i].find("a"):
            url1="https://www.espncricinfo.com/"+tdata[i].find("a").get('href')

            r1=requests.get(url1)

            inside= BeautifulSoup(r1.text,"html.parser")

            name1 = inside.find("div", class_="ds-bg-raw-black/85")
            # print(name1)
            # print(name1.find("h1","ds-text-title-l ds-font-bold").text)
            temp.append(name1.find("h1","ds-text-title-l ds-font-bold").text)
            temp.append(name1.find_all("span","ds-text-comfortable-s")[0].text)
            temp.append(name1.find_all("span","ds-text-comfortable-s")[1].text)
            # temp.append(name1.find("span","ds-text-comfortable-s").text)
        else:
            temp.append(tdata[i].text)
    data.append(temp)




In [ ]:
body=table.find("tbody")
rows = body.select("tr.ds-table-row-compact-bottom, tr.ds-table-row-compact-top")

data = []
for row in rows:
    temp = []
    tdata = row.find_all("td", class_="ds-min-w-max")
    for td in tdata:
        print(td.text)
        temp.append(td.text)

    data.append(temp)


SA Abbott
2023-2023
1
-
0
-
-
-
0.00
-
-
-
Australia
Abdullah Shafique
2023-2023
8
8
0
336
113
42.00
93.33
1
3
1
Pakistan
CN Ackermann
2023-2023
9
9
0
216
69
24.00
82.12
-
1
1
Netherlands
Agha Salman
2023-2023
3
1
0
51
51
51.00
113.33
-
1
-
Pakistan
MM Ali
2023-2023
6
6
0
95
42
15.83
74.80
-
-
-
England
KIC Asalanka
2023-2023
9
8
0
288
108
36.00
85.20
1
1
-
Sri Lanka
R Ashwin
2023-2023
1
-
0
-
-
-
0.00
-
-
-
India
AAP Atkinson
2023-2023
3
3
1
37
35
18.50
160.86
-
-
1
England
Azmatullah Omarzai
2023-2023
9
8
3
353
97*
70.60
97.78
-
3
-
Afghanistan
Babar Azam
2023-2023
9
9
1
320
74
40.00
82.90
-
4
-
Pakistan
JM Bairstow
2023-2023
9
9
0
215
59
23.88
88.47
-
2
1
England
W Barresi
2023-2023
4
4
0
83
41
20.75
74.10
-
-
-
Netherlands
T Bavuma
2023-2023
8
8
0
145
35
18.12
73.60
-
-
1
South Africa
TA Boult
2023-2023
10
4
3
21
10*
21.00
84.00
-
-
-
New Zealand
HC Brook
2023-2023
6
6
0
169
66
28.16
112.66
-
1
-
England
JJ Bumrah
2023-2023
11
3
1
18
16
9.00
62.06
-
-
-
India
JC Buttler
2023-2023
9

In [15]:
data

[['Sean Abbott',
  'Australia',
  'Bowling Allrounder',
  '2023-2023',
  '1',
  '1',
  '60',
  '-',
  '61',
  '2',
  '2/61',
  '30.50',
  '6.10',
  '30.0',
  '-',
  '-',
  '1',
  '-'],
 ['Abdullah Shafique',
  'Pakistan',
  'Top order Batter',
  '2023-2023',
  '8',
  '-',
  '0',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '2',
  '-'],
 ['Colin Ackermann',
  'Netherlands',
  'Batting Allrounder',
  '2023-2023',
  '9',
  '9',
  '284',
  '2',
  '234',
  '5',
  '2/39',
  '46.80',
  '4.94',
  '56.8',
  '-',
  '-',
  '3',
  '-'],
 ['Agha Salman',
  'Pakistan',
  'Allrounder',
  '2023-2023',
  '3',
  '2',
  '30',
  '-',
  '46',
  '-',
  '-',
  '-',
  '9.20',
  '-',
  '-',
  '-',
  '3',
  '-'],
 ['Moeen Ali',
  'England',
  'Batting Allrounder',
  '2023-2023',
  '6',
  '6',
  '256',
  '-',
  '248',
  '5',
  '3/42',
  '49.60',
  '5.81',
  '51.2',
  '-',
  '-',
  '4',
  '-'],
 ['Charith Asalanka',
  'Sri Lanka',
  'Batting Allrounder',
  '2023-2023',
  '9',
  '1',
  '6',
  '

In [16]:
head

['Player',
 'Span',
 'Mat',
 'Inns',
 'Balls',
 'Mdns',
 'Runs',
 'Wkts',
 'BBI',
 'Ave',
 'Econ',
 'SR',
 '5',
 '10',
 'Ct',
 'St']

In [17]:
data[0]

['Sean Abbott',
 'Australia',
 'Bowling Allrounder',
 '2023-2023',
 '1',
 '1',
 '60',
 '-',
 '61',
 '2',
 '2/61',
 '30.50',
 '6.10',
 '30.0',
 '-',
 '-',
 '1',
 '-']

In [18]:
df = pd.DataFrame(data, columns=[['Player','Country','Role',
 'Span',
 'Mat',
 'Inns',
 'Balls',
 'Mdns',
 'Runs',
 'Wkts',
 'BBI',
 'Ave',
 'Econ',
 'SR',
 '5',
 '10',
 'Ct',
 'St']])

In [19]:
df

,Player,Country,Role,Span,Mat,Inns,Balls,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,5,10,Ct,St
0,Sean Abbott,Australia,Bowling Allrounder,2023-2023,1,1,60,-,61,2,2/61,30.50,6.10,30.0,-,-,1,-
1,Abdullah Shafique,Pakistan,Top order Batter,2023-2023,8,-,0,-,-,-,-,-,-,-,-,-,2,-
2,Colin Ackermann,Netherlands,Batting Allrounder,2023-2023,9,9,284,2,234,5,2/39,46.80,4.94,56.8,-,-,3,-
3,Agha Salman,Pakistan,Allrounder,2023-2023,3,2,30,-,46,-,-,-,9.20,-,-,-,3,-
4,Moeen Ali,England,Batting Allrounder,2023-2023,6,6,256,-,248,5,3/42,49.60,5.81,51.2,-,-,4,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Chris Woakes,England,Allrounder,2023-2023,8,8,330,1,298,10,4/54,29.80,5.41,33.0,-,-,3,-
147,Mark Wood,England,Bowler,2023-2023,7,7,324,1,349,6,2/50,58.16,6.46,54.0,-,-,1,-
148,Suryakumar Yadav,India,Batter,2023-2023,7,1,12,-,17,-,-,-,8.50,-,-,-,1,-
149,Will Young,New Zealand,Top order Batter,2023-2023,6,-,0,-,-,-,-,-,-,-,-,-,2,-


In [20]:
df.to_csv('bowling_data.csv',index=False)